# Supervised Fine-Tuning (SFT) with Serverless customization on SageMaker AI

## Lab 2 - Fine-Tune LLM

In this notebook, we are going to run a Serverless Supervised Fine-Tuning job on SageMaker AI

***

### Prerequistes

#### Setup and dependencies

In [ ]:
import boto3
from sagemaker.core.helper.session_helper import Session, get_execution_role

sess = Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

s3_client = boto3.client("s3")
sess = Session(default_bucket=sagemaker_session_bucket)
sm_client = boto3.client("sagemaker", region_name=sess.boto_region_name)
bucket_name = sess.default_bucket()
default_prefix = sess.default_bucket_prefix

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
from sagemaker.ai_registry.dataset import DataSet

base_model_id = "huggingface-llm-qwen2-5-7b-instruct"

training_dataset = DataSet.get(name="medical-o1-reasoning-sft-train")
val_dataset = DataSet.get(name="medical-o1-reasoning-sft-val")

if default_prefix:
    output_path = f"s3://{bucket_name}/{default_prefix}/{base_model_id}"
else:
    output_path = f"s3://{bucket_name}/{base_model_id}"

os.environ["SAGEMAKER_MLFLOW_CUSTOM_ENDPOINT"] = (
    f"https://mlflow.sagemaker.{sess.boto_region_name}.app.aws"
)

***

### Create Model Package Group

In [ ]:
from botocore.exceptions import ClientError

model_package_group_name = f"{base_model_id}-mpg"
model_package_group_description = ""

operation_input_args = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": model_package_group_description,
}

try:
    sm_client.create_model_package_group(**operation_input_args)
except ClientError as e:
    if e.response["Error"]["Code"] == "ValidationException":
        print("Model Package Group already exists")
    else:
        raise

### Run Severless Job

In [ ]:
from sagemaker.train.common import TrainingType
from sagemaker.train.sft_trainer import SFTTrainer

In [ ]:
trainer = SFTTrainer(
    model=base_model_id,
    training_type=TrainingType.LORA,
    model_package_group=model_package_group_name,
    training_dataset=training_dataset,
    validation_dataset=val_dataset,
    s3_output_path=output_path,
    sagemaker_session=sess,
    role=role,
    accept_eula=True,
)

Print Hyperparameters

In [ ]:
from rich import print as rprint
from rich.pretty import pprint

print("Default Finetuning options:")
pprint(trainer.hyperparameters.to_dict())

Override Hyperparameters

In [ ]:
trainer.hyperparameters.learning_rate = 0.0001
trainer.hyperparameters.global_batch_size = 64
trainer.hyperparameters.max_epochs = 3
trainer.hyperparameters.lr_warmup_ratio = 0.1

In [ ]:
print("\nModified/user defined options:")
pprint(trainer.hyperparameters.to_dict())

In [ ]:
from rich import print as rprint
from rich.pretty import pprint

training_job = trainer.train(wait=False)

TRAINING_JOB_NAME = training_job.training_job_name

pprint(training_job)